In [1]:
from os import listdir
from os.path import isfile, join
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.plot import show
from fastai.data.all import *
from fastai.vision.all import *
#from fastai.vision.image import *
import fastbook
import fastai
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
import shutil
from PIL import Image
from matplotlib.pyplot import imshow
import torch
from torch.overrides import *
from torch.nn.functional import *
from torch.nn.modules.loss import _WeightedLoss
from torch.nn import *
#from IPython.core.debugg

In [2]:
path=r"C:\Users\Almut Eberhardt\Desktop\Bachelor_Notebook\Test_IM"

In [3]:
#copied from https://github.com/fastai/fastai/blob/f633356359a29f8d869ce36659f7aa25660e946a/fastai/data/transforms.py#L230
class Categorize_OneHot(DisplayedTransform):
    "Reversible transform of category string to `vocab` id"
    loss_func,order=CrossEntropyLossFlat(),1
    def __init__(self, vocab=None, sort=True, add_na=False):
        if vocab is not None: vocab = CategoryMap(vocab, sort=sort, add_na=add_na)
        store_attr()

    def setups(self, dsets):
        if self.vocab is None and dsets is not None: self.vocab = CategoryMap(dsets, sort=self.sort, add_na=self.add_na)
        self.c = len(self.vocab)

    def encodes(self, o):
        try:
            #change to one-hot-encoding
            
            length=len(self.vocab.o2i)
            y_vec=torch.zeros(length)
            y_vec[self.vocab.o2i[o]]=1
            return TensorCategory(y_vec.type(torch.LongTensor))#TensorCategory(self.vocab.o2i[o])
        except KeyError as e:
            raise KeyError(f"Label '{o}' was not included in the training dataset") from e
    #change
    def decodes(self, o): 
        index=torch.argmax(o)
        return Category(self.vocab[index])

In [4]:
images=DataBlock(
    blocks=(ImageBlock,TransformBlock(type_tfms=Categorize_OneHot(vocab=None, sort=True, add_na=False))),
    #blocks=(ImageBlock,CategoryBlock),
    #TransformBlock(type_tfms=Categorize_OneHot(vocab=None, sort=True, add_na=False))),
    #TransformBlock(type_tfms=Categorize32f(vocab=None, sort=True, add_na=False))),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label)

In [5]:
dls=images.dataloaders(path,num_workers=0,batch_size=4)

In [6]:
class Test_Net(nn.Module):
    def __init__(self):
        super(Test_Net,self).__init__()
        self.Conv1=nn.Conv2d(3,50,kernel_size=3,padding=1)
        self.ReLU1=nn.ReLU()
        self.Pool1=nn.MaxPool2d(kernel_size=2)
        self.Conv2=nn.Conv2d(50,24,kernel_size=3,padding=1)
        self.ReLU2=nn.ReLU()
        self.Pool2=nn.MaxPool2d(kernel_size=2)
        self.Conv3=nn.Conv2d(24,4,kernel_size=3,padding=1)
        self.ReLU3=nn.ReLU()
        self.Pool3=nn.MaxPool2d(kernel_size=2)
        #print(type(self.Pool3))
        self.Flat1=Flatten()
        self.Dense1=nn.Linear(144,100)
        self.ReLU4=nn.ReLU()
        self.Dense2=nn.Linear(100,2)
        self.ReLU5=nn.ReLU()
        self.Softmax=nn.Softmax(dim=1)
        #self.Flat2=Flatten()
    def forward(self,x):
        print("forward")
        res=self.Conv1(x)
        res=self.ReLU1(res)
        res=self.Pool1(res)
        res=self.Conv2(res)
        res=self.ReLU2(res)
        res=self.Pool2(res)
        res=self.Conv3(res)
        res=self.ReLU3(res)
        res=self.Pool3(res)
        res=self.Flat1(res)
        print(res.shape)
        res=self.Dense1(res)
        res=self.ReLU4(res)
        res=self.Dense2(res)
        res=self.Softmax(res)
        #res=self.Flat2(res)
        #res=self.ReLU5(res)
        return res

In [7]:
class BCELoss_mod(_WeightedLoss):
    r"""Creates a criterion that measures the Binary Cross Entropy
    between the target and the output:

    The unreduced (i.e. with :attr:`reduction` set to ``'none'``) loss can be described as:

    .. math::
        \ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
        l_n = - w_n \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right],

    where :math:`N` is the batch size. If :attr:`reduction` is not ``'none'``
    (default ``'mean'``), then

    .. math::
        \ell(x, y) = \begin{cases}
            \operatorname{mean}(L), & \text{if reduction} = \text{'mean';}\\
            \operatorname{sum}(L),  & \text{if reduction} = \text{'sum'.}
        \end{cases}

    This is used for measuring the error of a reconstruction in for example
    an auto-encoder. Note that the targets :math:`y` should be numbers
    between 0 and 1.

    Notice that if :math:`x_n` is either 0 or 1, one of the log terms would be
    mathematically undefined in the above loss equation. PyTorch chooses to set
    :math:`\log (0) = -\infty`, since :math:`\lim_{x\to 0} \log (x) = -\infty`.
    However, an infinite term in the loss equation is not desirable for several reasons.

    For one, if either :math:`y_n = 0` or :math:`(1 - y_n) = 0`, then we would be
    multiplying 0 with infinity. Secondly, if we have an infinite loss value, then
    we would also have an infinite term in our gradient, since
    :math:`\lim_{x\to 0} \frac{d}{dx} \log (x) = \infty`.
    This would make BCELoss's backward method nonlinear with respect to :math:`x_n`,
    and using it for things like linear regression would not be straight-forward.

    Our solution is that BCELoss clamps its log function outputs to be greater than
    or equal to -100. This way, we can always have a finite loss value and a linear
    backward method.


    Args:
        weight (Tensor, optional): a manual rescaling weight given to the loss
            of each batch element. If given, has to be a Tensor of size `nbatch`.
        size_average (bool, optional): Deprecated (see :attr:`reduction`). By default,
            the losses are averaged over each loss element in the batch. Note that for
            some losses, there are multiple elements per sample. If the field :attr:`size_average`
            is set to ``False``, the losses are instead summed for each minibatch. Ignored
            when reduce is ``False``. Default: ``True``
        reduce (bool, optional): Deprecated (see :attr:`reduction`). By default, the
            losses are averaged or summed over observations for each minibatch depending
            on :attr:`size_average`. When :attr:`reduce` is ``False``, returns a loss per
            batch element instead and ignores :attr:`size_average`. Default: ``True``
        reduction (string, optional): Specifies the reduction to apply to the output:
            ``'none'`` | ``'mean'`` | ``'sum'``. ``'none'``: no reduction will be applied,
            ``'mean'``: the sum of the output will be divided by the number of
            elements in the output, ``'sum'``: the output will be summed. Note: :attr:`size_average`
            and :attr:`reduce` are in the process of being deprecated, and in the meantime,
            specifying either of those two args will override :attr:`reduction`. Default: ``'mean'``

    Shape:
        - Input: :math:`(N, *)` where :math:`*` means, any number of additional
          dimensions
        - Target: :math:`(N, *)`, same shape as the input
        - Output: scalar. If :attr:`reduction` is ``'none'``, then :math:`(N, *)`, same
          shape as input.

    Examples::

        >>> m = nn.Sigmoid()
        >>> loss = nn.BCELoss()
        >>> input = torch.randn(3, requires_grad=True)
        >>> target = torch.empty(3).random_(2)
        >>> output = loss(m(input), target)
        >>> output.backward()
    """
    __constants__ = ['reduction']

    def __init__(self, weight: Optional[Tensor] = None, size_average=None, reduce=None, reduction: str = 'mean') -> None:
        super(BCELoss_mod, self).__init__(weight, size_average, reduce, reduction)

    def forward(self, input: Tensor, target: Tensor) -> Tensor:
        #print(input.shape)
        #target=target[:,None]
        target=target.type(torch.float)
        #print(target.shape)
        return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


In [8]:
tn=Test_Net()

In [9]:
def accuracy_mod(inp, targ, axis=-1):
    "Compute accuracy with `targ` when `pred` is bs * n_classes"
    #print(inp)
    #print(targ.shape)
    #pred,targ = flatten_check(inp.argmax(dim=axis), targ)
    pred=inp.argmax(dim=axis)
    targ=targ.argmax(dim=axis)
    #print(pred)
    return (pred == targ).float().mean()

In [10]:
learn = Learner(dls, tn, loss_func=BCELoss_mod(), metrics=accuracy_mod)
#learn.model=learn.model.cpu()

In [11]:
learn.fit_one_cycle(10, 0.001)

epoch,train_loss,valid_loss,accuracy_mod,time
0,0.694299,0.698448,0.475000,00:01
1,0.693210,0.683879,0.625000,00:00
2,0.582953,0.295347,1.000000,00:00
3,0.319401,0.022572,1.000000,00:00
4,0.145535,0.004863,1.000000,00:00
5,0.065652,0.003663,1.000000,00:00
6,0.029993,0.003471,1.000000,00:00
7,0.014013,0.003511,1.000000,00:00
8,0.006856,0.003372,1.000000,00:00
9,0.003642,0.003362,1.000000,00:00


forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(40, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 144)
forward
(4, 1